# Chapter 5. Polars (Part 1)

## Topics covered:

<a href="http://nbviewer.jupyter.org/github/RandyBetancourt/PythonForSASUsers/blob/master/Chapter%2004%20--%20polars%2C%20Part%201.ipynb#Importing-Packages">Importing Packages</a>

<a href="http://nbviewer.jupyter.org/github/RandyBetancourt/PythonForSASUsers/blob/master/Chapter%2004%20--%20polars%2C%20Part%201.ipynb#Series">Series</a>

<a href="http://nbviewer.jupyter.org/github/RandyBetancourt/PythonForSASUsers/blob/master/Chapter%2004%20--%20polars%2C%20Part%201.ipynb#DataFrames"> DataFrames </a>

<a href="http://nbviewer.jupyter.org/github/RandyBetancourt/PythonForSASUsers/blob/master/Chapter%2004%20--%20polars%2C%20Part%201.ipynb#pd.read_csv()"> Read .csv files </a>

<a href="http://nbviewer.jupyter.org/github/RandyBetancourt/PythonForSASUsers/blob/master/Chapter%2004%20--%20polars%2C%20Part%201.ipynb#Insepction"> Inspection </a>

<a href="http://nbviewer.jupyter.org/github/RandyBetancourt/PythonForSASUsers/blob/master/Chapter%2004%20--%20polars%2C%20Part%201.ipynb#Handling-Missing-Data"> Handling Missing Data </a>

<a href="http://nbviewer.jupyter.org/github/RandyBetancourt/PythonForSASUsers/blob/master/Chapter%2004%20--%20polars%2C%20Part%201.ipynb#Missing-Value-Identification"> Missing Data Detection</a>

<a href="http://nbviewer.jupyter.org/github/RandyBetancourt/PythonForSASUsers/blob/master/Chapter%2004%20--%20polars%2C%20Part%201.ipynb#Missing-Value-Replacement"> Missing Value Replacement </a>

<a href="http://nbviewer.jupyter.org/github/RandyBetancourt/PythonForSASUsers/blob/master/Chapter%2004%20--%20polars%2C%20Part%201.ipynb#Resources"> Resources </a>


This chapter introduces the `polars` library (or package). Polars is built using Rust rather than NumPy, and it exposes a fast, expressive API for data manipulation in Python.

Up until this point, the examples in this notebook have used Python’s built-in types and functions. NumPy provides the ndarray object for numerical array computations and remains the foundation of much scientific computing in Python. We will illustrate a few useful NumPy objects as a point of comparison, but our primary focus will now shift to Polars.

Polars was developed to support high-performance data analysis with much more flexibility than raw arrays provide. In real data-analysis tasks, we frequently need to work with heterogeneous types in the same table—such as strings for categories, integers for counts, and floats for continuous measures. In addition, we often want labeled columns, grouping operations, joins, pivots, time-series features, and other relational operations.

Polars provides two essential data structures: the Series and the DataFrame.
A Series represents a one-dimensional, typed array with an index-like position, roughly analogous to a column of data.
A DataFrame represents a two-dimensional table, conceptually indexed by rows and columns—very similar to a spreadsheet or database relation.

In other words, a Polars DataFrame resembles a SAS data set (or a relational database table).
The table below compares Polars components to those found in SAS.

<table border="1">
  <tr>
    <td> <b>Polars </b></td>
    <td> <b>SAS </b></td>
  </tr>
    <td> DataFrame </td>
    <td> SAS data set </td>
  </tr>
  <tr>
    <td> row </td>
    <td> observation </td>
  </tr>
  <tr>
    <td> column </td>
    <td> variable </td>
  </tr>
  <tr>
    <td> group_by </td>
    <td> BY-Group </td>
  </tr>
  <tr>
    <td> NaN </td>
    <td><b>.</b> </td>
  </tr>
    <tr>
    <td> slice </td>
    <td> sub-set </td>
  </tr>
    <tr>
    <td> axis 0 / rows </td>
    <td>observation</td>
  </tr>
    </tr>
    <tr>
    <td> axis 1 / columns </td>
    <td>column</td>
  </tr>
    </tr>
    <tr>
    <td> filter </td>
    <td>where</td>
  </tr>
</table>

#### polars Pre-requisites

polars has two main data structures:

    1. Series
    2. DataFrame


## Importing Packages

To begin utilizing `polars` objects, or other objects, begin by importing libraies by name into our namespace.  To avoid having to re-typing package name repeatedly, use the standard aliases of np for NumPy and `pl` for polars.

In [1]:
import polars as pl
from numpy.random import randn
from IPython.display import Image

## Series
A Series can be thought of as a one-dimensional array with labels.  This structure includes an index of labels used as keys to locate values.  Data in a Series can be any data type.  polars data types are covered in detail <a href="http://nbviewer.jupyter.org/github/RandyBetancourt/PythonForSASUsers/blob/master/Chapter%2003%20%20--%20Data%20Types%20and%20Formatting.ipynb"> here </a>.  In the SAS examples, we use Data Step ARRAYs as an analog to the Series.

Start by creating a Series of random values.

In [2]:
s1 = pl.Series(randn(10))
print(s1.head(5))

shape: (5,)
Series: '' [f64]
[
	-0.066529
	-1.037518
	1.443927
	-1.677282
	1.390976
]


In [3]:
# get the first element
s1[0]  # 0-based

-0.0665287723627356

Notice when indexing, the start position begins with 0.  Most SAS automatic variables like \_n\_ use 1 as the index start position.   Iteration of the SAS DO loop 0 to 9 in conjunction with an ARRAY produces an array subscript out of range error.

In the SAS example below the DO loop is used to iterate over the array elements locating the target elements. 

Arrays in SAS are used primarily for iteratively processing like variables together.  SAS/IML is a closer analog to NumPy arrays.  <a href="http://support.sas.com/documentation/cdl/en/imlug/68150/HTML/default/viewer.htm#imlug_imlstart_toc.htm"> SAS/IML </a>is outside the scope of these examples.

````
    /******************************************************/
    /* c04_array_random_values.sas                        */
    /******************************************************/
    4         data _null_;
    5         
    6         call streaminit(54321);
    7         
    8         array s2 {10} ;
    9         do i = 1 to 10;
    10           s2{i} = rand("Uniform");
    11        
    12           if i <= 5 then put
    13              s2{i};
    14        end;

    0.4322317772
    0.5977982976
    0.7785986473
    0.1748250183
    0.3941470125
````

Return the first 3 elements in the Series.

In [4]:
print(s1[:3])

shape: (3,)
Series: '' [f64]
[
	-0.066529
	-1.037518
	1.443927
]


````
    /******************************************************/
    /* c04_return_first_3_array_elements.sas              */
    /******************************************************/
    20        data _null_;
    21        
    22        call streaminit(54321);
    23        
    24        array s2 {10} ;
    25        do i = 1 to 10;
    26           s2{i} = rand("Uniform");
    27        
    28           if i <= 3 then put
    29              s2{i};
    30        end;

    0.4322317772
    0.5977982976
    0.7785986473
````

The example has two operations.  The s2.mean() method calculates mean followed by a boolen test less than this calculated mean.

In [5]:
# filter is like the 'where' clause
s1.filter(s1 < s1.mean())

""
f64
-0.066529
-1.037518
-1.677282
-0.746996
-0.808957


Series and other objects have attributes using a dot (.) chaining-style syntax.  .name is one a number of attributes for the Series object.

In [6]:
s1.name  # no name, this will print '' which is an empty string

''

In [7]:
s2 = s1.rename('Arbitrary Name')
print(s2.head(5))

shape: (5,)
Series: 'Arbitrary Name' [f64]
[
	-0.066529
	-1.037518
	1.443927
	-1.677282
	1.390976
]


Polars dataframes and series are immutable, meaning that they can't change. Thus, when we call `rename`, we're creating a new dataframe. This is returned and we assign it to the variable `s2`. `s1` still references the previous dataframe.

In [8]:
print(s1.head(5))  # no name

shape: (5,)
Series: '' [f64]
[
	-0.066529
	-1.037518
	1.443927
	-1.677282
	1.390976
]


## DataFrames

DataFrames are relational-like structures with labels.  Alternatively, a DataFrame with a single column is a Series.

Like SAS, DataFrames have different methods for creation.  DataFrames can be created by loading values from other Python objects.  Data values can also be loaded from a range of non-Python input sources, including .csv files, DBMS tables, Web API's, and even SAS data sets (.sas7bdat), etc.  Details are discussed in [Chapter 11](10-Polars_Readers.ipynb)

Start by reading the UK_Accidents .csv file.  It contains vehicular accident data in the U.K from January 1, 2015 to December 31, 2015.  The .csv file is located [here](data/uk_accidents.csv).

There are multiple reports for each day of the year.  The values are mostly integer values using the Road-Accident_Safety-Data_Guide.xls file found [here](data/Road-Accident-Safety-Data-Guide.xls) to map values to descriptive labels.

## pl.read_csv()

The default values are used in the example below.  polars provide a number of readers having parameters for controling missing values, date parsing, line skipping, data type mapping, etc.  These parameters are analogous to SAS' INFILE/INPUT processing.

Additional examples of reading various data inputs into a DataFrame are covered in <a href="http://nbviewer.jupyter.org/github/RandyBetancourt/PythonForSASUsers/blob/master/Chapter%2011%20--%20polars%20Readers.ipynb">Chapter 11 -- polars Readers</a>

We'll place a `r` before the string so that the backslashes are read in the Windows filename.

In [9]:
from pathlib import Path  # use this library for path management
filepath = Path(r'data\uk_accidents.csv')
df = pl.read_csv(filepath)
df.head()

Accident_Severity,Number_of_Vehicles,Number_of_Casualties,Day_of_Week,Time,Road_Type,Speed_limit,Junction_Detail,Light_Conditions,Weather_Conditions,Road_Surface_Conditions,Urban_or_Rural_Area,Vehicle_Reference,Vehicle_Type,Skidding_and_Overturning,Was_Vehicle_Left_Hand_Drive_,Sex_of_Driver,Age_of_Driver,Engine_Capacity__CC_,Propulsion_Code,Age_of_Vehicle,Casualty_Class,Sex_of_Casualty,Age_of_Casualty,Casualty_Severity,Car_Passenger,Date
i64,i64,i64,i64,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str
3,2,1,6,"""19:00""",3,40,1,4,1,1,1,1,9,0,1,1,30,-1,-1,-1,1,1,54,3,0,"""1/9/2015"""
3,2,1,6,"""19:00""",3,40,1,4,1,1,1,2,9,0,1,1,54,1499,2,1,1,1,54,3,0,"""1/9/2015"""
3,3,1,2,"""18:30""",3,40,1,4,2,2,1,1,9,0,1,1,30,-1,-1,-1,1,2,20,3,0,"""2/23/2015"""
3,3,1,2,"""18:30""",3,40,1,4,2,2,1,2,9,0,1,2,20,1199,1,13,1,2,20,3,0,"""2/23/2015"""
3,3,1,2,"""18:30""",3,40,1,4,2,2,1,3,9,0,1,1,30,-1,-1,-1,1,2,20,3,0,"""2/23/2015"""


PROC IMPORT is used to read the same .csv file.  This is one of several methods for SAS to read a .csv file.  Here we have taken the defaults.

````
    /******************************************************/
    /* c04_read_csv_proc_import.sas                       */
    /******************************************************/
    5  proc import datafile='c:\data\uk_accidents.csv' out=uk_accidents;
     
    NOTE: The file 'c:\data\uk_accidents.csv' is:
          File Name 'c:\data\uk_accidents.csv',
          Lrecl=32760, Recfm=V

    NOTE: 266776 records were read from file 'c:\data\uk_accidents.csv'
          The minimum record length was 65
          The maximum record length was 77
    NOTE: Data set "WORK.uk_accidents" has 266776 observation(s) and 27 variable(s)
````

Unlike SAS, the Python interpreter is mainly silent upon normal execution.  When debugging it is helpful to invoke methods and functions to return information about these objects.  This is somewhat analogous to use PUT statements in the SAS log to examine variable values.

The size, shape, and ndim attributes report respectively, number of cells, rows/columns, and number of dimensions are shown below. 

In [10]:
print(f'Shape: {df.shape}')
print(f'Number of rows (height): {df.height}')
print(f'Number of columns (width): {df.width}')

Shape: (266776, 27)
Number of rows (height): 266776
Number of columns (width): 27


#### Read Verification

After reading a file, you often want to understand its content and structure.  The DataFrame `.describe()` method returns descriptions of the DataFrame's attributes.  In SAS, this same information is generally found in the output from PROC CONTENTS.

In [11]:
df.describe()

statistic,Accident_Severity,Number_of_Vehicles,Number_of_Casualties,Day_of_Week,Time,Road_Type,Speed_limit,Junction_Detail,Light_Conditions,Weather_Conditions,Road_Surface_Conditions,Urban_or_Rural_Area,Vehicle_Reference,Vehicle_Type,Skidding_and_Overturning,Was_Vehicle_Left_Hand_Drive_,Sex_of_Driver,Age_of_Driver,Engine_Capacity__CC_,Propulsion_Code,Age_of_Vehicle,Casualty_Class,Sex_of_Casualty,Age_of_Casualty,Casualty_Severity,Car_Passenger,Date
str,f64,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str
"""count""",266776.0,266776.0,266776.0,266776.0,"""266752""",266776.0,266776.0,266776.0,266776.0,266776.0,266776.0,266776.0,266776.0,266776.0,266776.0,266776.0,266776.0,266776.0,266776.0,266776.0,266776.0,266776.0,266776.0,266776.0,266776.0,266776.0,"""266776"""
"""null_count""",0.0,0.0,0.0,0.0,"""24""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"""0"""
"""mean""",2.871529,4.103994,2.53937,4.270043,null,4.724953,44.235801,1.908481,2.702897,1.775782,1.80385,1.457744,1.56465,9.573826,0.189582,0.997249,1.404958,35.454209,1377.679105,0.813068,5.179308,1.361892,1.46229,34.026303,2.883502,0.20094,null
"""std""",0.361889,4.388495,2.457659,1.757526,null,1.708642,17.679664,2.440738,2.163288,1.462043,1.170044,0.498641,0.762442,8.667575,0.727782,0.144418,0.605817,20.031944,1675.091608,1.342504,5.882103,0.645716,0.500018,18.60533,0.34356,0.518167,null
"""min""",1.0,1.0,1.0,1.0,"""0:01""",1.0,0.0,-1.0,1.0,1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,1.0,-1.0,"""1/1/2015"""
"""25%""",3.0,2.0,1.0,3.0,null,3.0,30.0,0.0,1.0,1.0,1.0,1.0,1.0,9.0,0.0,1.0,1.0,23.0,-1.0,-1.0,-1.0,1.0,1.0,19.0,3.0,0.0,null
"""50%""",3.0,2.0,1.0,5.0,null,6.0,30.0,0.0,1.0,1.0,1.0,1.0,1.0,9.0,0.0,1.0,1.0,35.0,1364.0,1.0,4.0,1.0,1.0,30.0,3.0,0.0,null
"""75%""",3.0,3.0,3.0,5.0,null,6.0,60.0,3.0,4.0,2.0,2.0,2.0,2.0,9.0,0.0,1.0,2.0,49.0,1896.0,2.0,10.0,2.0,2.0,47.0,3.0,0.0,null
"""max""",3.0,37.0,38.0,7.0,"""9:59""",9.0,70.0,9.0,7.0,9.0,5.0,2.0,37.0,98.0,5.0,2.0,3.0,97.0,22311.0,12.0,105.0,3.0,2.0,104.0,3.0,2.0,"""9/9/2015"""


## Inspection

polars have methods which used to inspect to data values.  The DataFrame .head() method displays the first 5 rows by default.  The .tail() method displays the last 5 rows by default.  The row count value can be an arbitrary integer value such as:

    # display the last 20 rows of the DataFrame
    df.tail(20)
    
SAS uses the FIRSTOBS and OBS options with procedures to determine input observations. The SAS code to print the last 20 observations of the uk_accidents data set is:

    proc print data=uk_accidents (firstobs = 266756);

In [12]:
df.head()

Accident_Severity,Number_of_Vehicles,Number_of_Casualties,Day_of_Week,Time,Road_Type,Speed_limit,Junction_Detail,Light_Conditions,Weather_Conditions,Road_Surface_Conditions,Urban_or_Rural_Area,Vehicle_Reference,Vehicle_Type,Skidding_and_Overturning,Was_Vehicle_Left_Hand_Drive_,Sex_of_Driver,Age_of_Driver,Engine_Capacity__CC_,Propulsion_Code,Age_of_Vehicle,Casualty_Class,Sex_of_Casualty,Age_of_Casualty,Casualty_Severity,Car_Passenger,Date
i64,i64,i64,i64,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str
3,2,1,6,"""19:00""",3,40,1,4,1,1,1,1,9,0,1,1,30,-1,-1,-1,1,1,54,3,0,"""1/9/2015"""
3,2,1,6,"""19:00""",3,40,1,4,1,1,1,2,9,0,1,1,54,1499,2,1,1,1,54,3,0,"""1/9/2015"""
3,3,1,2,"""18:30""",3,40,1,4,2,2,1,1,9,0,1,1,30,-1,-1,-1,1,2,20,3,0,"""2/23/2015"""
3,3,1,2,"""18:30""",3,40,1,4,2,2,1,2,9,0,1,2,20,1199,1,13,1,2,20,3,0,"""2/23/2015"""
3,3,1,2,"""18:30""",3,40,1,4,2,2,1,3,9,0,1,1,30,-1,-1,-1,1,2,20,3,0,"""2/23/2015"""


OBS=n in SAS determines the number of observations used as input.

````
    /******************************************************/
    /* c_04_display_1st_5_obs.sas                         */
    /******************************************************/
    39       proc print data = uk_accidents (obs=5);

    The output from PROC PRINT is not displayed here. 
````

Scoping output by columns is shown in the cell below. The column list is analogous to the VAR statement in PROC PRINT. The square braces ([]) is creating a list of the column names we want to look at.

In [13]:
df.select(['Sex_of_Driver', 'Time']).head(10)

Sex_of_Driver,Time
i64,str
1,"""19:00"""
1,"""19:00"""
1,"""18:30"""
2,"""18:30"""
1,"""18:30"""
1,"""17:50"""
1,"""17:50"""
1,"""7:05"""
1,"""7:05"""


Notice the DataFrame default index (incrementing from 0 to 9).  This is analogous to the SAS automatic variable \_n\_.  Later, we illustrate using other columns in the DataFrame as the index.  

Below is the SAS program to print the first 10 observations of a data set along with the variables 'sex_of_driver' and 'time'.

````
    /******************************************************/
    /* c04_scoping_obs_and_variables.sas                  */
    /******************************************************/
    40       proc print data = uk_accidents (obs=10);
    41          var sex_of_driver time;

    The output from PROC PRINT is not displayed here.
````

## Handling Missing Data

Before analyzing data a common task is dealing with missing data.  polars uses two designations to indicate missing data, NaN (not a number) and the Python None object.


## Missing Value Identification

Returning to our DataFrame, we need an analysis of missing values for all the columns.  polars provide four methods for the detection and replacement of missing values.  They are:

<table border="1">
  <tr>
    <td> <b>Method </b></td>
    <td> <b>Action Taken </b></td>
  </tr>
    <td> is_null() </td>
    <td> generates a boolean mask to indicate missing values </td>
  </tr>
  <tr>
    <td> is_not_null() </td>
    <td> opposite of is_null() </td>
  </tr>
  <tr>
    <td> drop_nulls() </td>
    <td> returns a filtered version of the data where any row with nulls is dropped </td>
  </tr>
  <tr>
    <td> fill_null(0) </td>
    <td> returns a copy of data with missing values filled (e.g., 0 above) or imputed </td>
  </tr>

</table>

We will look at each of these in detail below.  

A typical SAS-programming approach to address the missing data analysis is to write a program to traverses all columns using counter variables with IF/THEN testing for missing values.  

This can be along the lines of the example in cell #19 below.  df.columns returns the sequence of column names in the DataFrame.  

In [14]:
for col_name in df.columns:
    print(col_name, end="---->")
    print(sum(df[col_name].is_null()))

Accident_Severity---->0
Number_of_Vehicles---->0
Number_of_Casualties---->0
Day_of_Week---->0
Time---->24
Road_Type---->0
Speed_limit---->0
Junction_Detail---->0
Light_Conditions---->0
Weather_Conditions---->0
Road_Surface_Conditions---->0
Urban_or_Rural_Area---->0
Vehicle_Reference---->0
Vehicle_Type---->0
Skidding_and_Overturning---->0
Was_Vehicle_Left_Hand_Drive_---->0
Sex_of_Driver---->0
Age_of_Driver---->0
Engine_Capacity__CC_---->0
Propulsion_Code---->0
Age_of_Vehicle---->0
Casualty_Class---->0
Sex_of_Casualty---->0
Age_of_Casualty---->0
Casualty_Severity---->0
Car_Passenger---->0
Date---->0


While this give the desired results, there is a better approach.  

As an aside, if you find yourself thinking of solving a polars operation (or Python for that matter) using iterative processing, stop and take a little time to do research.  Chances are, a method or function already exists!

Case-in-point is illustrated below.  It chains the .sum() attribute to the .is_null() attribute to return a count of the missing values for the columns in the DataFrame.

The .is_null() method returns True for missing values.  By chaining the .sum() method to the .isnull() method it produces a count of the missing values for each columns.

In [15]:
df.select(
    # pl.all -> for each column, do: is_null().sum() (sum where is null)
    pl.all().is_null().sum()
)

Accident_Severity,Number_of_Vehicles,Number_of_Casualties,Day_of_Week,Time,Road_Type,Speed_limit,Junction_Detail,Light_Conditions,Weather_Conditions,Road_Surface_Conditions,Urban_or_Rural_Area,Vehicle_Reference,Vehicle_Type,Skidding_and_Overturning,Was_Vehicle_Left_Hand_Drive_,Sex_of_Driver,Age_of_Driver,Engine_Capacity__CC_,Propulsion_Code,Age_of_Vehicle,Casualty_Class,Sex_of_Casualty,Age_of_Casualty,Casualty_Severity,Car_Passenger,Date
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,24,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [17]:
# or, transpose this dataframe to get a betterview
df.select(
    # pl.all -> for each column, do: is_null().sum() (sum where is null)
    pl.all().is_null().sum()
).transpose(include_header=True)

column,column_0
str,u32
"""Accident_Severity""",0
"""Number_of_Vehicles""",0
"""Number_of_Casualties""",0
"""Day_of_Week""",0
"""Time""",24
…,…
"""Sex_of_Casualty""",0
"""Age_of_Casualty""",0
"""Casualty_Severity""",0


Not all of the rows are displayed. We can increase the limit for number of rows to show with:

In [18]:
pl.Config.set_tbl_rows(30)  # see up to 30 rows each time

polars.config.Config

In [19]:
# or, transpose this dataframe to get a betterview
df.select(
    # pl.all -> for each column, do: is_null().sum() (sum where is null)
    pl.all().is_null().sum()
).transpose(include_header=True)

column,column_0
str,u32
"""Accident_Severity""",0
"""Number_of_Vehicles""",0
"""Number_of_Casualties""",0
"""Day_of_Week""",0
"""Time""",24
"""Road_Type""",0
"""Speed_limit""",0
"""Junction_Detail""",0
"""Light_Conditions""",0


To identify missing values the SAS example below uses PROC Format to bin missing and non-missing values.  Missing values are represented by default as (*.*) for numeric and blank (' ') for character variables.  Therefore, a user-defined format is needed for both types.  

PROC FREQ is used with the automatic variables \_CHARACTER\_ and \_NUMERIC\_ to produce a frequency listing for each variable type.  

Only a portion of the SAS output is shown since separate output is produced for each variable.  As with the example in cell # 19 above, the 'time' variable is the only variable with missing values.

````
    /******************************************************/
    /* c04_find_missing_numerics_characters.sas           */
    /******************************************************/
    26       proc format;
    27        value $missfmt ' '='Missing' other='Not Missing';
    28        value  missfmt  . ='Missing' other='Not Missing';
    29       run;

    30       
    31       proc freq data=uk_accidents;
    32          format _CHARACTER_ $missfmt.;
    33          tables _CHARACTER_ / missing missprint nocum nopercent;
    34       
    35          format _NUMERIC_ missfmt.;
    36          tables _NUMERIC_ / missing missprint nocum nopercent;
````

Another method for detecting missing values is to search column-wise by using the any_horizontal() function to the chained attributes .isnull().any().  The operation is then performed along columns.

In [20]:
null_data = df.filter(
    pl.any_horizontal(pl.all().is_null())
)
null_data.head()  # dataframe with rows containing missing data from any column

Accident_Severity,Number_of_Vehicles,Number_of_Casualties,Day_of_Week,Time,Road_Type,Speed_limit,Junction_Detail,Light_Conditions,Weather_Conditions,Road_Surface_Conditions,Urban_or_Rural_Area,Vehicle_Reference,Vehicle_Type,Skidding_and_Overturning,Was_Vehicle_Left_Hand_Drive_,Sex_of_Driver,Age_of_Driver,Engine_Capacity__CC_,Propulsion_Code,Age_of_Vehicle,Casualty_Class,Sex_of_Casualty,Age_of_Casualty,Casualty_Severity,Car_Passenger,Date
i64,i64,i64,i64,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str
3,2,1,2,null,1,30,2,1,1,1,1,1,9,0,1,1,-1,2148,2,12,2,2,44,3,1,"""1/5/2015"""
3,1,1,4,null,6,30,0,1,2,2,1,3,9,0,1,1,51,-1,-1,-1,1,1,18,3,0,"""7/8/2015"""
3,2,1,5,null,6,30,0,1,1,1,1,1,9,0,1,2,43,-1,-1,-1,1,1,29,3,0,"""9/3/2015"""
3,1,1,3,null,6,60,9,1,8,4,2,3,9,0,1,2,48,1560,2,1,3,2,70,2,0,"""1/20/2015"""
3,2,1,7,null,1,30,1,4,1,1,1,1,5,1,1,1,54,749,1,17,1,2,21,3,0,"""1/24/2015"""


## Missing Value Replacement

The code below is used to render multiple objects side-by-side.  It is from  Essential Tools for Working With Data, by Jake VanderPlas found <a href="http://shop.oreilly.com/product/0636920034919.do"> here </a>.  It displays the 'before' and 'after' effects of changes to objects together.  

In [21]:
class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""

    def __init__(self, *args):
        self.args = args

    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)

    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)

To illustrate the .fillna() method, consider the following to create a DataFrame.  

In [24]:
df2 = pl.DataFrame({
    'temp': ['cold', 'warm', 'hot', 'cool', 'cool', 'cold'],
    'speed': ['slow', 'medium', 'fast', None, 'medium', 'slow'],
    'size': [None, 4, 9, None, 16, None],
    'height': [2., 5, 4, None, 44, 29],
    'power': [6., 7, None, 17, 21, 33],
    'age': [3., 9, 6, 89, 13, 17]
})

print(df2)
display("df2")

shape: (6, 6)
┌──────┬────────┬──────┬────────┬───────┬──────┐
│ temp ┆ speed  ┆ size ┆ height ┆ power ┆ age  │
│ ---  ┆ ---    ┆ ---  ┆ ---    ┆ ---   ┆ ---  │
│ str  ┆ str    ┆ i64  ┆ f64    ┆ f64   ┆ f64  │
╞══════╪════════╪══════╪════════╪═══════╪══════╡
│ cold ┆ slow   ┆ null ┆ 2.0    ┆ 6.0   ┆ 3.0  │
│ warm ┆ medium ┆ 4    ┆ 5.0    ┆ 7.0   ┆ 9.0  │
│ hot  ┆ fast   ┆ 9    ┆ 4.0    ┆ null  ┆ 6.0  │
│ cool ┆ null   ┆ null ┆ null   ┆ 17.0  ┆ 89.0 │
│ cool ┆ medium ┆ 16   ┆ 44.0   ┆ 21.0  ┆ 13.0 │
│ cold ┆ slow   ┆ null ┆ 29.0   ┆ 33.0  ┆ 17.0 │
└──────┴────────┴──────┴────────┴───────┴──────┘


temp,speed,size,height,power,age
str,str,i64,f64,f64,f64
"""cold""","""slow""",null,2.0,6.0,3.0
"""warm""","""medium""",4,5.0,7.0,9.0
"""hot""","""fast""",9,4.0,null,6.0
"""cool""",null,null,null,17.0,89.0
"""cool""","""medium""",16,44.0,21.0,13.0
"""cold""","""slow""",null,29.0,33.0,17.0


In [25]:
# get count of nulls per column
df_tf = df2.select(pl.all().is_null().sum())
display("df2", "df_tf")

temp,speed,size,height,power,age
str,str,i64,f64,f64,f64
"""cold""","""slow""",null,2.0,6.0,3.0
"""warm""","""medium""",4,5.0,7.0,9.0
"""hot""","""fast""",9,4.0,null,6.0
"""cool""",null,null,null,17.0,89.0
"""cool""","""medium""",16,44.0,21.0,13.0
"""cold""","""slow""",null,29.0,33.0,17.0
temp,speed,size,height,power,age
u32,u32,u32,u32,u32,u32
0,1,3,1,1,0


By default the .drop_nulls() method drops either the entire row or column in which any null value is found.

In [27]:
df3 = df2.drop_nulls()
display("df2", "df3")

temp,speed,size,height,power,age
str,str,i64,f64,f64,f64
"""cold""","""slow""",null,2.0,6.0,3.0
"""warm""","""medium""",4,5.0,7.0,9.0
"""hot""","""fast""",9,4.0,null,6.0
"""cool""",null,null,null,17.0,89.0
"""cool""","""medium""",16,44.0,21.0,13.0
"""cold""","""slow""",null,29.0,33.0,17.0
temp,speed,size,height,power,age
str,str,i64,f64,f64,f64
"""warm""","""medium""",4,5.0,7.0,9.0


The .drop_nulls() method also works by choosing the columns we want to exclude if they contani nulls: `subset`.

In [28]:
# drop only rows with size/height being null
df4 = df2.drop_nulls(subset=['size', 'height'])
display("df2", "df4")

temp,speed,size,height,power,age
str,str,i64,f64,f64,f64
"""cold""","""slow""",null,2.0,6.0,3.0
"""warm""","""medium""",4,5.0,7.0,9.0
"""hot""","""fast""",9,4.0,null,6.0
"""cool""",null,null,null,17.0,89.0
"""cool""","""medium""",16,44.0,21.0,13.0
"""cold""","""slow""",null,29.0,33.0,17.0
temp,speed,size,height,power,age
str,str,i64,f64,f64,f64
"""warm""","""medium""",4,5.0,7.0,9.0


Rather than dropping rows and columns, missing values can be imputed or replaced.  The .fillna() method returns either a Series or a DataFrame with null values replaced.  The example below replaces all NaN's with zero.

In [30]:
df6 = df2.fill_null(0)
display("df2", "df6")

temp,speed,size,height,power,age
str,str,i64,f64,f64,f64
"""cold""","""slow""",null,2.0,6.0,3.0
"""warm""","""medium""",4,5.0,7.0,9.0
"""hot""","""fast""",9,4.0,null,6.0
"""cool""",null,null,null,17.0,89.0
"""cool""","""medium""",16,44.0,21.0,13.0
"""cold""","""slow""",null,29.0,33.0,17.0
temp,speed,size,height,power,age
str,str,i64,f64,f64,f64
"""cold""","""slow""",0,2.0,6.0,3.0


An imputation method based on the mean value of df['col6'] is shown below.  The .fillna() method finds and then replaces all occurences of NaN with this calculated value.

In [34]:
df8 = df2.select(["age", "size", "height"]).fill_null(df2['age'].mean())
display("df2", "df8")

df2
shape: (6, 6)
┌──────┬────────┬──────┬────────┬───────┬──────┐
│ temp ┆ speed  ┆ size ┆ height ┆ power ┆ age  │
│ ---  ┆ ---    ┆ ---  ┆ ---    ┆ ---   ┆ ---  │
│ str  ┆ str    ┆ i64  ┆ f64    ┆ f64   ┆ f64  │
╞══════╪════════╪══════╪════════╪═══════╪══════╡
│ cold ┆ slow   ┆ null ┆ 2.0    ┆ 6.0   ┆ 3.0  │
│ warm ┆ medium ┆ 4    ┆ 5.0    ┆ 7.0   ┆ 9.0  │
│ hot  ┆ fast   ┆ 9    ┆ 4.0    ┆ null  ┆ 6.0  │
│ cool ┆ null   ┆ null ┆ null   ┆ 17.0  ┆ 89.0 │
│ cool ┆ medium ┆ 16   ┆ 44.0   ┆ 21.0  ┆ 13.0 │
│ cold ┆ slow   ┆ null ┆ 29.0   ┆ 33.0  ┆ 17.0 │
└──────┴────────┴──────┴────────┴───────┴──────┘

df8
shape: (6, 3)
┌──────┬───────────┬───────────┐
│ age  ┆ size      ┆ height    │
│ ---  ┆ ---       ┆ ---       │
│ f64  ┆ f64       ┆ f64       │
╞══════╪═══════════╪═══════════╡
│ 3.0  ┆ 22.833333 ┆ 2.0       │
│ 9.0  ┆ 4.0       ┆ 5.0       │
│ 6.0  ┆ 9.0       ┆ 4.0       │
│ 89.0 ┆ 22.833333 ┆ 22.833333 │
│ 13.0 ┆ 16.0      ┆ 44.0      │
│ 17.0 ┆ 22.833333 ┆ 29.0      │
└──────┴───────────┴───────────┘

The corresponding SAS program is shown below.  The PROC SQL SELECT INTO clause stores the calculated mean for the variable "col6" into the macro variable &col6_mean.  This is followed by a Data Step iterating the array 'x' for "col3 - col5" replacing missing values with &col6_mean. 

A more detailed example of replacing missing values with group means is located <a href="http://nbviewer.jupyter.org/github/RandyBetancourt/PythonForSASUsers/blob/master/Chapter%2007%20--%20polars%2C%20Part%202.ipynb#Replace-Missing-Values-with-Group-Means"> here</a>. 

SAS/Stat has PROC MI for imputation of missing values with a range of methods described <a href="https://support.sas.com/documentation/cdl/en/statug/63962/HTML/default/viewer.htm#statug_mi_sect001.htm">
here.</a>  PROC MI is outside the scope of these examples.

````
    /******************************************************/
    /* c04_replace_missing_with_mean_values.sas           */
    /******************************************************/
    4         data df;
    5         infile cards dlm=',';
    6         
    7         input col1 $
    8               col2 $
    9               col3
    10              col4
    11              col5
    12              col6 ;
    13        
    14        datalines;
    15        cold, slow, ., 2, 6, 3
    16        warm, medium, 4, 5, 7, 9
    17        hot, fast, 9, 4, ., 6
    18        cool, , ., ., 17, 89
    19        cool,  medium, 16, 44, 21, 13
    20        cold, slow, . ,29, 33, 17
    21        ;;;;

    22        proc sql;
    23           select mean(col6) into :col6_mean
    24           from df;
    25        quit;

    26        
    27        data df2;
    28           set df;
    29         array x {3} col3-col5  ;
    30        
    31         do i = 1 to 3;
    32            if x(i) = . then x(i) = &col6_mean;
    33         end;
````

The fill_null(strategy='forward') is a 'forward' fill method.  NaN's are replaced by the adjacent cell above traversing 'down' the columns.  Cell #32 below constrasts the DataFrame df2, created in cell #24 above with the DataFrame df9 created with the 'forward' fill method.

In [35]:
df9 = df2.fill_null(strategy='forward')
display("df2", "df9")

temp,speed,size,height,power,age
str,str,i64,f64,f64,f64
"""cold""","""slow""",null,2.0,6.0,3.0
"""warm""","""medium""",4,5.0,7.0,9.0
"""hot""","""fast""",9,4.0,null,6.0
"""cool""",null,null,null,17.0,89.0
"""cool""","""medium""",16,44.0,21.0,13.0
"""cold""","""slow""",null,29.0,33.0,17.0
temp,speed,size,height,power,age
str,str,i64,f64,f64,f64
"""cold""","""slow""",null,2.0,6.0,3.0


Simalarly, the .fill_null(strategy='backward') is a 'backwards' fill method.  NaN's are replaced by the adjecent cell traversing 'up' the columns.  Cell #32 constrasts the  DataFrame df2, created in cell #23 above with the DataFrame df10 created with the 'backward' fill method.

In [36]:
df10 = df2.fill_null(strategy='backward')
display("df2", "df10")

temp,speed,size,height,power,age
str,str,i64,f64,f64,f64
"""cold""","""slow""",null,2.0,6.0,3.0
"""warm""","""medium""",4,5.0,7.0,9.0
"""hot""","""fast""",9,4.0,null,6.0
"""cool""",null,null,null,17.0,89.0
"""cool""","""medium""",16,44.0,21.0,13.0
"""cold""","""slow""",null,29.0,33.0,17.0
temp,speed,size,height,power,age
str,str,i64,f64,f64,f64
"""cold""","""slow""",4,2.0,6.0,3.0


Cell #34 contrasts DataFrame df9 created in cell #32 using the 'forward' fill method with DataFrame df10 created in cell #33 with the 'backward' fill method.

In [37]:
display("df9", "df10")

temp,speed,size,height,power,age
str,str,i64,f64,f64,f64
"""cold""","""slow""",null,2.0,6.0,3.0
"""warm""","""medium""",4,5.0,7.0,9.0
"""hot""","""fast""",9,4.0,7.0,6.0
"""cool""","""fast""",9,4.0,17.0,89.0
"""cool""","""medium""",16,44.0,21.0,13.0
"""cold""","""slow""",16,29.0,33.0,17.0
temp,speed,size,height,power,age
str,str,i64,f64,f64,f64
"""cold""","""slow""",4,2.0,6.0,3.0


Before dropping the missing rows, calculate the portion of records lost in the accidents DataFrame, df created above.

In [39]:
n_null = df['Time'].is_null().sum()
pct_null = n_null / df.height
print(f"{n_null} records in the DataFrame will be dropped.")
print(f'The portion of records dropped is {pct_null:6.3%}')

24 records in the DataFrame will be dropped.
The portion of records dropped is 0.009%


The .drop_null() method is silent except in the case of errors.  We can verify the DataFrame's shape after the method is applied.

In [42]:
print(df.height)
print(df.drop_nulls().height)

266776
266752


# Resources

<a href="http://byumcl.bitbucket.org/bootcamp2013/labs/polars.html"> polars: Data Handling and Analysis in Python </a> from 2013 BYU MCL Bootcamp documentation.

<a href="http://www.gregreda.com/2013/10/26/intro-to-polars-data-structures/"> Intro to polars data structures </a> by Greg Reda.  This is a three-part series using the Movie Lens data set nicely to illustrate polars.

<a href="http://www.webpages.uidaho.edu/~stevel/504/polars%20DataFrame%20Notes.pdf"> Cheat Sheet: The polars DataFrame Object </a>by Mark Graph and located at the University of Idaho's web-site.

<a href="http://polars.pydata.org/polars-docs/stable/missing_data.html"> Working with missing data </a> polars documentation.

## Navigation

* [Next Chapter](06-Polars.ipynb)
* [Return to Contents](00-Contents.ipynb)